# Preparing Input Data

Win rates need to be aggregated across different sources:
1. Newer Logs
    1. `data_processing/refined.csv` - LLMs as Black playing against Random Player. The key data source for all models to be rated and included in the LB
    2. `data_processing/dragon_refined.csv` - LLMs as Black playing against different levels of Komodo Dragon chess engine (Elo rated against chess.com pool)
2. Older Logs
    1. `_logs/_pre_aug_2025/dragon_vs_llm` - LLMs as Black playing against different levels of Komodo Dragon chess engine (Elo rated against chess.com pool)
    2. `_logs/_pre_aug_2025/misc/dragon` - different levels of dragon playing different colors vs Random Player and Stockfish
    3. `_logs/_pre_aug_2025/llm_vs_llm` - LLMs vs LLMs

## Dataset Structure

- player_white - model name or one of the values: 'random_player', 'dragon-lvl-X' (where X is integer in range from 1 to 25), 'stockfish-lvl-X' (where X is integer in range from 0 to 20)
- player_black - same as white values, denotes all games as black
- games - total games the given white/black couple had
- win_rate_white - percent of the games won by the white player
- win_rate_black - same for black
- draw_rate - percent of Draws
- player_black_elo - only valid for Dragon levels (e.g. lvl 1 is 250, lvl 2 is 375, lvl 3 is 500 etc.), empty for other (non rated) players
- player_white_elo - same as above
- source - indicates which data source this row came from: 'refined', 'dragon_refined', 'dragon_vs_llm', 'misc_dragon', 'llm_vs_llm'


## Reusable/Globals

In [44]:
import csv
import json
from pathlib import Path
import re
from collections import defaultdict
from prettytable import PrettyTable
from tabulate import tabulate

# Paths
REFINED_CSV = Path(".") / "refined.csv"
DRAGON_REFINED_CSV = Path(".") / "dragon_refined.csv"
WIN_RATES_CSV = Path(".") / "win_rates.csv"
DRAGON_VS_LLM_DIR = Path("..") / "_logs" / "_pre_aug_2025" / "dragon_vs_llm"
MISC_DRAGON_DIR = Path("..") / "_logs" / "_pre_aug_2025" / "misc" / "dragon"
LLM_VS_LLM_DIR = Path("..") / "_logs" / "_pre_aug_2025" / "llm_vs_llm"

# Prepare output header as per dataset specification
HEADER = [
    "player_white",
    "player_black",
    "games",
    "win_rate_white",
    "win_rate_black",
    "draw_rate",
    "player_white_elo",
    "player_black_elo",
    "source",
]

CURR_ROWS = 0

ROWS_OUT = []

# Counters per source
refined_count = 0
dragon_refined_count = 0
dragon_vs_llm_count = 0
misc_dragon_count = 0
llm_vs_llm_count = 0

# Helper functions
def dragon_elo(level):
    """Calculate Dragon Elo: lvl 1 = 250, lvl 2 = 375, etc."""
    return 250 + 125 * (level - 1)

def parse_dragon_level(name):
    """Extract dragon level from name like 'dragon-lvl-1' or 'lvl-1'"""
    match = re.search(r'lvl-(\d+)', name)
    return int(match.group(1)) if match else None

def read_aggregate_json(json_path):
    """Read _aggregate_results.json and return game stats"""
    with open(json_path, 'r') as f:
        data = json.load(f)
    return {
        'total_games': data['total_games'],
        'white_wins': data['white_wins'],
        'black_wins': data['black_wins'],
        'draws': data['draws']
    }

def compute_win_rates(white_wins, black_wins, draws, total_games):
    """Compute win rates as percentages"""
    if white_wins + black_wins + draws != total_games:
        raise ValueError(f"Inconsistent game counts: {white_wins} + {black_wins} + {draws} != {total_games}")
    
    win_rate_white = round(white_wins * 100.0 / total_games, 3)
    win_rate_black = round(black_wins * 100.0 / total_games, 3)
    draw_rate = round(draws * 100.0 / total_games, 3)
    
    return win_rate_white, win_rate_black, draw_rate

def preview_win_rates(rows_or_csv, preview_top=3, preview_bottom=5):
    """
    Print a pretty table preview of the win rates.
    Accepts either a path to a CSV file, a list of rows (with header as first row),
    or a list of dicts (all with identical keys).
    Shows top N rows, ellipsis if more than (top+bottom), and bottom M rows.
    Returns (rows, header).
    """
    # Determine if input is a path or already rows
    if hasattr(rows_or_csv, "open"):  # likely a Path
        with rows_or_csv.open("r", newline="") as f:
            reader = list(csv.reader(f))
            header = reader[0]
            rows = reader[1:]
    elif isinstance(rows_or_csv, list):
        if not rows_or_csv:
            print("No data to preview.")
            return [], []
        if isinstance(rows_or_csv[0], dict):
            header = list(rows_or_csv[0].keys())
            rows = [[row.get(col, "") for col in header] for row in rows_or_csv]
        else:
            header = rows_or_csv[0]
            rows = rows_or_csv[1:]
    else:
        raise ValueError("Input must be a Path, a list of rows (with header as first row), or a list of dicts")

    table = PrettyTable()
    table.field_names = header

    # Show top N rows if exists
    for i in range(min(preview_top, len(rows))):
        table.add_row(rows[i])

    # Add three dots row if there are more than (top+bottom) rows
    if len(rows) > (preview_top + preview_bottom):
        table.add_row(['...'] * len(header))

    # Add bottom M rows (or all remaining if less than M)
    for row in rows[-preview_bottom:]:
        table.add_row(row)

    print(table)
    return rows, header


## Ingesting Data

### 1. 2. `refined.csv`, ### 2. `dragon_refined.csv`

In [45]:
# Ingest refined.csv (LLM as Black vs Random Player)
with REFINED_CSV.open("r", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        total_games = int(row["total_games"])
        player_wins = int(row["player_wins"])
        opponent_wins = int(row["opponent_wins"])
        draws = int(row["draws"])

        # Basic sanity check
        if player_wins + opponent_wins + draws != total_games:
            raise ValueError(
                f"Row for player {row['Player']} has inconsistent win/loss/draw counts"
            )

        # Compute win rates as percentages (0-100)
        win_rate_white = round(opponent_wins * 100.0 / total_games, 3)
        win_rate_black = round(player_wins * 100.0 / total_games, 3)
        draw_rate = round(draws * 100.0 / total_games, 3)

        ROWS_OUT.append(
            {
                "player_white": "random_player",
                "player_black": row["Player"],
                "games": total_games,
                "win_rate_white": win_rate_white,
                "win_rate_black": win_rate_black,
                "draw_rate": draw_rate,
                "player_white_elo": "",
                "player_black_elo": "",
                "source": "refined",
            }
        )
        refined_count += 1

print(f"Loaded {refined_count} rows from refined.csv")

preview_win_rates(ROWS_OUT)


Loaded 122 rows from refined.csv
+---------------+------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+---------+
|  player_white |            player_black            | games | win_rate_white | win_rate_black | draw_rate | player_white_elo | player_black_elo |  source |
+---------------+------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+---------+
| random_player |        gpt-5-2025-08-07-low        |   43  |      0.0       |     95.349     |   4.651   |                  |                  | refined |
| random_player |     gpt-5-mini-2025-08-07-high     |   35  |      0.0       |     88.571     |   11.429  |                  |                  | refined |
| random_player |         o3-2025-04-16-low          |   42  |     2.381      |     90.476     |   7.143   |                  |                  | refined |
|      ...      |        

([['random_player',
   'gpt-5-2025-08-07-low',
   43,
   0.0,
   95.349,
   4.651,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-high',
   35,
   0.0,
   88.571,
   11.429,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-low',
   42,
   2.381,
   90.476,
   7.143,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-medium',
   33,
   3.03,
   90.909,
   6.061,
   '',
   '',
   'refined'],
  ['random_player',
   'o4-mini-2025-04-16-high',
   39,
   2.564,
   89.744,
   7.692,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-high',
   33,
   3.03,
   87.879,
   9.091,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-medium',
   33,
   0.0,
   81.818,
   18.182,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-medium',
   53,
   9.434,
   90.566,
   0.0,
   '',
   '',
   'refined'],
  ['random_player',
   'o1-2024-12-17-medium',
   41,
   2.439,
   8

In [46]:
# Ingest dragon_refined.csv (LLMs as Black vs Dragon levels)
if DRAGON_REFINED_CSV.exists():
    with DRAGON_REFINED_CSV.open("r", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            total_games = int(row["total_games"])
            player_wins = int(row["player_wins"])  # LLM (Black) wins
            opponent_wins = int(row["opponent_wins"])  # Dragon (White) wins
            draws = int(row["draws"])

            # Sanity check
            if player_wins + opponent_wins + draws != total_games:
                raise ValueError(
                    f"Row for player {row['Player']} has inconsistent counts"
                )

            # The opponent in these rows is a dragon level (column name has a known typo: 'white_opponent')
            white_name = (
                row.get("white_opponent")
                or row.get("white_opponent")
                or row.get("opponent")
                or row.get("Opponent")
                or ""
            )
            dragon_level = parse_dragon_level(white_name)
            player_name = row.get("Player") or row.get("player")

            # Compute win rates
            win_rate_white = round(opponent_wins * 100.0 / total_games, 3)  # Dragon as White
            win_rate_black = round(player_wins * 100.0 / total_games, 3)    # LLM as Black
            draw_rate = round(draws * 100.0 / total_games, 3)

            # Resolve player_white label and Elo
            player_white = f"dragon-lvl-{dragon_level}" if dragon_level is not None else white_name
            player_white_elo = dragon_elo(dragon_level) if dragon_level is not None else ""

            ROWS_OUT.append(
                {
                    "player_white": player_white,
                    "player_black": player_name,
                    "games": total_games,
                    "win_rate_white": win_rate_white,
                    "win_rate_black": win_rate_black,
                    "draw_rate": draw_rate,
                    "player_white_elo": player_white_elo,
                    "player_black_elo": "",
                    "source": "dragon_refined",
                }
            )
            dragon_refined_count += 1

print(f"Loaded {dragon_refined_count} rows from dragon_refined.csv")

preview_win_rates(ROWS_OUT)


Loaded 43 rows from dragon_refined.csv
+---------------+-------------------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+----------------+
|  player_white |                   player_black                  | games | win_rate_white | win_rate_black | draw_rate | player_white_elo | player_black_elo |     source     |
+---------------+-------------------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+----------------+
| random_player |               gpt-5-2025-08-07-low              |   43  |      0.0       |     95.349     |   4.651   |                  |                  |    refined     |
| random_player |            gpt-5-mini-2025-08-07-high           |   35  |      0.0       |     88.571     |   11.429  |                  |                  |    refined     |
| random_player |                o3-2025-04-16-low                |   42  | 

([['random_player',
   'gpt-5-2025-08-07-low',
   43,
   0.0,
   95.349,
   4.651,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-high',
   35,
   0.0,
   88.571,
   11.429,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-low',
   42,
   2.381,
   90.476,
   7.143,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-medium',
   33,
   3.03,
   90.909,
   6.061,
   '',
   '',
   'refined'],
  ['random_player',
   'o4-mini-2025-04-16-high',
   39,
   2.564,
   89.744,
   7.692,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-high',
   33,
   3.03,
   87.879,
   9.091,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-medium',
   33,
   0.0,
   81.818,
   18.182,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-medium',
   53,
   9.434,
   90.566,
   0.0,
   '',
   '',
   'refined'],
  ['random_player',
   'o1-2024-12-17-medium',
   41,
   2.439,
   8

In [47]:
with REFINED_CSV.open("r", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        total_games = int(row["total_games"])
        player_wins = int(row["player_wins"])
        opponent_wins = int(row["opponent_wins"])
        draws = int(row["draws"])

        # Basic sanity check
        if player_wins + opponent_wins + draws != total_games:
            raise ValueError(
                f"Row for player {row['Player']} has inconsistent win/loss/draw counts"
            )

        # Compute win rates as percentages (0-100)
        win_rate_white = round(opponent_wins * 100.0 / total_games, 3)
        win_rate_black = round(player_wins * 100.0 / total_games, 3)
        draw_rate = round(draws * 100.0 / total_games, 3)

        ROWS_OUT.append(
            {
                "player_white": "random_player",
                "player_black": row["Player"],
                "games": total_games,
                "win_rate_white": win_rate_white,
                "win_rate_black": win_rate_black,
                "draw_rate": draw_rate,
                "player_white_elo": "",
                "player_black_elo": "",
                "source": "refined",
            }
        )

print(f"Loaded {len(ROWS_OUT)} rows from refined.csv")

preview_win_rates(ROWS_OUT)


Loaded 287 rows from refined.csv
+---------------+------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+---------+
|  player_white |            player_black            | games | win_rate_white | win_rate_black | draw_rate | player_white_elo | player_black_elo |  source |
+---------------+------------------------------------+-------+----------------+----------------+-----------+------------------+------------------+---------+
| random_player |        gpt-5-2025-08-07-low        |   43  |      0.0       |     95.349     |   4.651   |                  |                  | refined |
| random_player |     gpt-5-mini-2025-08-07-high     |   35  |      0.0       |     88.571     |   11.429  |                  |                  | refined |
| random_player |         o3-2025-04-16-low          |   42  |     2.381      |     90.476     |   7.143   |                  |                  | refined |
|      ...      |        

([['random_player',
   'gpt-5-2025-08-07-low',
   43,
   0.0,
   95.349,
   4.651,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-high',
   35,
   0.0,
   88.571,
   11.429,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-low',
   42,
   2.381,
   90.476,
   7.143,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-medium',
   33,
   3.03,
   90.909,
   6.061,
   '',
   '',
   'refined'],
  ['random_player',
   'o4-mini-2025-04-16-high',
   39,
   2.564,
   89.744,
   7.692,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-nano-2025-08-07-high',
   33,
   3.03,
   87.879,
   9.091,
   '',
   '',
   'refined'],
  ['random_player',
   'gpt-5-mini-2025-08-07-medium',
   33,
   0.0,
   81.818,
   18.182,
   '',
   '',
   'refined'],
  ['random_player',
   'o3-2025-04-16-medium',
   53,
   9.434,
   90.566,
   0.0,
   '',
   '',
   'refined'],
  ['random_player',
   'o1-2024-12-17-medium',
   41,
   2.439,
   8

### 2. `_logs/dragon_vs_llm` - Dragon (White) vs LLM (Black)


In [48]:
# Process dragon_vs_llm directory
dragon_vs_llm_count = 0

if DRAGON_VS_LLM_DIR.exists():
    for pair_dir in DRAGON_VS_LLM_DIR.iterdir():
        if not pair_dir.is_dir():
            continue
        
        # Skip error/fail directories
        if pair_dir.name.startswith(('errors-', 'fails-')):
            continue
        
        # Parse directory name: lvl-X_vs_Y
        if '_vs_' not in pair_dir.name:
            continue
        
        white_part, black_part = pair_dir.name.split('_vs_', 1)
        
        # Extract dragon level
        dragon_level = parse_dragon_level(white_part)
        if dragon_level is None:
            continue
        
        # Aggregate across all timestamp directories
        total_games = 0
        total_white_wins = 0
        total_black_wins = 0
        total_draws = 0
        
        for timestamp_dir in pair_dir.iterdir():
            if not timestamp_dir.is_dir():
                continue
            
            aggregate_file = timestamp_dir / "_aggregate_results.json"
            if aggregate_file.exists():
                stats = read_aggregate_json(aggregate_file)
                total_games += stats['total_games']
                total_white_wins += stats['white_wins']
                total_black_wins += stats['black_wins']
                total_draws += stats['draws']
        
        # Skip if no games found
        if total_games == 0:
            continue
        
        # Compute win rates
        win_rate_white, win_rate_black, draw_rate = compute_win_rates(
            total_white_wins, total_black_wins, total_draws, total_games
        )
        
        # Add row
        ROWS_OUT.append({
            "player_white": f"dragon-lvl-{dragon_level}",
            "player_black": black_part,
            "games": total_games,
            "win_rate_white": win_rate_white,
            "win_rate_black": win_rate_black,
            "draw_rate": draw_rate,
            "player_white_elo": dragon_elo(dragon_level),
            "player_black_elo": "",
            "source": "dragon_vs_llm",
        })
        
        dragon_vs_llm_count += 1

print(f"Loaded {dragon_vs_llm_count} rows from dragon_vs_llm")


Loaded 28 rows from dragon_vs_llm


### 3. `_logs/misc/dragon` - Pre-aggregated Dragon vs Stockfish/Random


In [49]:
# Process misc/dragon directory
misc_dragon_count = 0

if MISC_DRAGON_DIR.exists():
    for json_file in MISC_DRAGON_DIR.glob("*.json"):
        # Parse filename: white_vs_black.json
        filename = json_file.stem
        if '_vs_' not in filename:
            continue
        
        white_name, black_name = filename.split('_vs_', 1)
        
        # Read the aggregated data
        stats = read_aggregate_json(json_file)
        
        # Compute win rates
        win_rate_white, win_rate_black, draw_rate = compute_win_rates(
            stats['white_wins'], stats['black_wins'], stats['draws'], stats['total_games']
        )
        
        # Determine Elo ratings
        white_elo = ""
        black_elo = ""
        
        # Check if white is dragon
        white_dragon_level = parse_dragon_level(white_name)
        if white_dragon_level is not None:
            white_elo = dragon_elo(white_dragon_level)
        
        # Check if black is dragon
        black_dragon_level = parse_dragon_level(black_name)
        if black_dragon_level is not None:
            black_elo = dragon_elo(black_dragon_level)
        
        # Add row
        ROWS_OUT.append({
            "player_white": white_name,
            "player_black": black_name,
            "games": stats['total_games'],
            "win_rate_white": win_rate_white,
            "win_rate_black": win_rate_black,
            "draw_rate": draw_rate,
            "player_white_elo": white_elo,
            "player_black_elo": black_elo,
            "source": "misc_dragon",
        })
        
        misc_dragon_count += 1

print(f"Loaded {misc_dragon_count} rows from misc/dragon")


Loaded 20 rows from misc/dragon


### 4. `_logs/llm_vs_llm` - LLM vs LLM


In [50]:
# Process llm_vs_llm directory
llm_vs_llm_count = 0

if LLM_VS_LLM_DIR.exists():
    for pair_dir in LLM_VS_LLM_DIR.iterdir():
        if not pair_dir.is_dir():
            continue
        
        # Parse directory name: white_vs_black
        if '_vs_' not in pair_dir.name:
            continue
        
        white_name, black_name = pair_dir.name.split('_vs_', 1)
        
        # Aggregate across all timestamp directories
        total_games = 0
        total_white_wins = 0
        total_black_wins = 0
        total_draws = 0
        
        for timestamp_dir in pair_dir.iterdir():
            if not timestamp_dir.is_dir():
                continue
            
            aggregate_file = timestamp_dir / "_aggregate_results.json"
            if aggregate_file.exists():
                stats = read_aggregate_json(aggregate_file)
                total_games += stats['total_games']
                total_white_wins += stats['white_wins']
                total_black_wins += stats['black_wins']
                total_draws += stats['draws']
        
        # Skip if no games found
        if total_games == 0:
            continue
        
        # Compute win rates
        win_rate_white, win_rate_black, draw_rate = compute_win_rates(
            total_white_wins, total_black_wins, total_draws, total_games
        )
        
        # Add row (no Elo for LLM vs LLM)
        ROWS_OUT.append({
            "player_white": white_name,
            "player_black": black_name,
            "games": total_games,
            "win_rate_white": win_rate_white,
            "win_rate_black": win_rate_black,
            "draw_rate": draw_rate,
            "player_white_elo": "",
            "player_black_elo": "",
            "source": "llm_vs_llm",
        })
        
        llm_vs_llm_count += 1

print(f"Loaded {llm_vs_llm_count} rows from llm_vs_llm")


Loaded 11 rows from llm_vs_llm


## Data Analysis and Model Name Standardization


In [51]:
# Collect all unique model names from all sources
all_model_names = set()

for row in ROWS_OUT:
    # Skip non-LLM players
    if row['player_white'] not in ['random_player'] and not row['player_white'].startswith(('dragon-lvl-', 'stockfish-lvl-')):
        all_model_names.add(row['player_white'])
    if row['player_black'] not in ['random_player'] and not row['player_black'].startswith(('dragon-lvl-', 'stockfish-lvl-')):
        all_model_names.add(row['player_black'])

print(f"Found {len(all_model_names)} unique model names across all sources:")
print("=" * 60)

# Group by data source
models_by_source = defaultdict(set)
for row in ROWS_OUT:
    source = row['source']
    if row['player_white'] not in ['random_player'] and not row['player_white'].startswith(('dragon-lvl-', 'stockfish-lvl-')):
        models_by_source[source].add(row['player_white'])
    if row['player_black'] not in ['random_player'] and not row['player_black'].startswith(('dragon-lvl-', 'stockfish-lvl-')):
        models_by_source[source].add(row['player_black'])

for source, models in models_by_source.items():
    print(f"\n{source.upper()} ({len(models)} models):")
    for model in sorted(models):
        print(f"  - {model}")

print("\n" + "=" * 60)


Found 147 unique model names across all sources:

REFINED (122 models):
  - amazon.nova-lite-v1
  - amazon.nova-pro-v1
  - chat-bison-32k@002
  - claude-v3-5-haiku
  - claude-v3-5-sonnet-v1
  - claude-v3-5-sonnet-v2
  - claude-v3-7-sonnet
  - claude-v3-7-sonnet-thinking_10000
  - claude-v3-7-sonnet-thinking_1024
  - claude-v3-7-sonnet-thinking_2048
  - claude-v3-7-sonnet-thinking_5000
  - claude-v3-haiku
  - claude-v3-opus
  - claude-v4-opus
  - claude-v4-opus-thinking_16000
  - claude-v4-sonnet
  - claude-v4-sonnet-thinking_16000
  - deephermes-3-llama-3-8b-preview@q8
  - deepseek-chat-v3
  - deepseek-chat-v3-0324
  - deepseek-r1-distill-qwen-14b@q8_0
  - deepseek-r1-distill-qwen-32b@q4_k_m
  - deepseek-reasoner-r1
  - gemini-1.5-flash-001
  - gemini-1.5-pro-preview-0409
  - gemini-2.0-flash-001
  - gemini-2.0-flash-exp
  - gemini-2.0-flash-lite-001
  - gemini-2.0-flash-lite-preview-02-05
  - gemini-2.0-flash-thinking-exp-01-21
  - gemini-2.0-flash-thinking-exp-1219
  - gemini-2.5-fla

In [52]:
# Analyze potential duplicates/variants
print("POTENTIAL MODEL NAME VARIANTS:")
print("=" * 60)

# Group similar names (simple heuristic based on common substrings)
model_groups = defaultdict(list)
for model in sorted(all_model_names):
    # Extract base model name (before @, -, or other separators)
    base_name = model.split('@')[0].split('-')[0].split('_')[0]
    model_groups[base_name].append(model)

# Show groups with multiple variants
for base_name, variants in model_groups.items():
    if len(variants) > 1:
        print(f"\n{base_name.upper()} variants:")
        for variant in variants:
            # Show which sources contain this variant
            sources = []
            for source, models in models_by_source.items():
                if variant in models:
                    sources.append(source)
            print(f"  - {variant} (sources: {', '.join(sources)})")

# Check for exact duplicates across sources
print("\n" + "=" * 60)
print("EXACT DUPLICATES ACROSS SOURCES:")
print("=" * 60)

for model in sorted(all_model_names):
    sources_with_model = []
    for source, models in models_by_source.items():
        if model in models:
            sources_with_model.append(source)
    
    if len(sources_with_model) > 1:
        print(f"{model}: {', '.join(sources_with_model)}")

print("\n" + "=" * 60)


POTENTIAL MODEL NAME VARIANTS:

3X variants:
  - 3x-o4-mini-2025-04-16-low_41mini-t03 (sources: dragon_vs_llm)
  - 3x-o4-mini-2025-04-16-low_o4-mini-2025-04-16-medium (sources: dragon_vs_llm)

4O variants:
  - 4o_mini (sources: llm_vs_llm)
  - 4o_mini_messaed_player_names (sources: llm_vs_llm)

AMAZON.NOVA variants:
  - amazon.nova-lite-v1 (sources: refined)
  - amazon.nova-pro-v1 (sources: refined)

CLAUDE variants:
  - claude-3-7-sonnet-20250219-thinking-budget-10000 (sources: dragon_refined, dragon_vs_llm)
  - claude-3-7-sonnet-20250219-thinking-budget-5000 (sources: dragon_refined, dragon_vs_llm)
  - claude-v3-5-haiku (sources: refined)
  - claude-v3-5-sonnet-v1 (sources: refined)
  - claude-v3-5-sonnet-v2 (sources: refined)
  - claude-v3-7-sonnet (sources: refined)
  - claude-v3-7-sonnet-thinking_10000 (sources: refined)
  - claude-v3-7-sonnet-thinking_1024 (sources: refined)
  - claude-v3-7-sonnet-thinking_2048 (sources: refined)
  - claude-v3-7-sonnet-thinking_5000 (sources: ref

In [53]:
# Create a standardization mapping
# This will be a manual mapping that we can adjust based on the analysis above
model_name_mapping = {
    # Add mappings here based on the analysis
    # Example: 'variant_name': 'standard_name'
}

# Function to standardize model names
def standardize_model_name(name):
    """Apply standardization mapping to model names"""
    return model_name_mapping.get(name, name)

# Apply standardization to all rows
standardized_rows = []
for row in ROWS_OUT:
    new_row = row.copy()
    new_row['player_white'] = standardize_model_name(row['player_white'])
    new_row['player_black'] = standardize_model_name(row['player_black'])
    standardized_rows.append(new_row)

print(f"Applied standardization mapping to {len(standardized_rows)} rows")
print(f"Mapping rules: {len(model_name_mapping)}")

# Check if any standardization was applied
changes_made = sum(1 for i, row in enumerate(ROWS_OUT) 
                  if (standardized_rows[i]['player_white'] != row['player_white'] or 
                      standardized_rows[i]['player_black'] != row['player_black']))

print(f"Rows with changes: {changes_made}")

# Use standardized rows for output
ROWS_OUT = standardized_rows


Applied standardization mapping to 346 rows
Mapping rules: 0
Rows with changes: 0


## Final Output


In [54]:
# Ensure deterministic ordering by player_white, then player_black
ROWS_OUT.sort(key=lambda x: (x["player_white"].lower(), x["player_black"].lower()))

# Write to CSV (overwrite)
with WIN_RATES_CSV.open("w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=HEADER)
    writer.writeheader()
    writer.writerows(ROWS_OUT)

print(f"\nTotal rows written to {WIN_RATES_CSV}: {len(ROWS_OUT)}")
print(f"- From refined.csv: {len(ROWS_OUT) - dragon_vs_llm_count - misc_dragon_count - llm_vs_llm_count}")
print(f"- From dragon_vs_llm: {dragon_vs_llm_count}")
print(f"- From misc/dragon: {misc_dragon_count}")
print(f"- From llm_vs_llm: {llm_vs_llm_count}")

# Show first 5 rows of the final output
print("\nFirst 5 rows of final output:")

# Convert first 5 rows to list of lists for tabulate
table_data = []
for row in ROWS_OUT[:5]:
    table_data.append([row[col] for col in HEADER])

print(tabulate(table_data, headers=HEADER, tablefmt="grid"))




Total rows written to win_rates.csv: 346
- From refined.csv: 287
- From dragon_vs_llm: 28
- From misc/dragon: 20
- From llm_vs_llm: 11

First 5 rows of final output:
+----------------+-----------------------------------------------------+---------+------------------+------------------+-------------+--------------------+--------------------+---------------+
| player_white   | player_black                                        |   games |   win_rate_white |   win_rate_black |   draw_rate | player_white_elo   | player_black_elo   | source        |
+================+=====================================================+=========+==================+==================+=============+====================+====================+===============+
| 4o_mini        | flash_2                                             |      30 |           26.667 |           70     |       3.333 |                    |                    | llm_vs_llm    |
+----------------+-------------------------------------------

## Elo

### Core Elo model

Let \(R\) be the rating of a player and \(R_{\text{opp}}\) the opponent’s rating. The expected score for the player under the Elo model is the logistic function (base 10):

\[ E = \nfrac{1}{1 + 10^{(R_{\text{opp}} - R)/400}} = \sigma\!\left(\frac{R_{\text{opp}} - R}{400/\log_{10}e}\right). \]

Scores are: win = 1, draw = 0.5, loss = 0.

### Handling draws and aggregated results

For \(N\) games against a specific opponent, if we observed wins \(W\), draws \(D\), and losses \(L\), the score fraction is

\[ S = \frac{W + 0.5\,D}{N}, \quad N = W + D + L. \]

When only aggregate win/draw/loss rates are available, we convert them to counts (or keep them as fractions with weights) before fitting.

### Color advantage (white advantage)

White has a measurable advantage. We model this as a constant \(\gamma\) Elo points (default \(\gamma = 35\)). We handle it by shifting the opponent’s effective rating depending on the player’s color:

- If the player is Black vs a White opponent: \(R^{\text{eff}}_{\text{opp}} = R_{\text{opp}} + \gamma\).
- If the player is White vs a Black opponent: \(R^{\text{eff}}_{\text{opp}} = R_{\text{opp}} - \gamma\).

Then we compute the expected score as \(E = 1/(1 + 10^{(R^{\text{eff}}_{\text{opp}} - R)/400})\).

### Quick closed-form for a single opponent

If a player faced a single rated opponent \(R_{\text{opp}}\) and achieved score fraction \(S\), the implied rating difference \(\Delta\) is

\[ \Delta = -400\,\log_{10}\!\left(\frac{1}{S} - 1\right). \]

Converting \(\Delta\) into the player’s rating requires color adjustment:

- Player as White: \(R = R_{\text{opp}} + \Delta - \gamma\).
- Player as Black: \(R = R_{\text{opp}} + \Delta + \gamma\).

This is a useful approximation when there is only one opponent.

### Maximum-likelihood estimate (MLE) with multiple opponents

With multiple rated opponents \(k\), each with \(n_k\) games and observed score fraction \(s_k\), define the opponent’s effective rating \(R^{\text{eff}}_{\text{opp},k}\) using the color rule above. The MLE \(\hat R\) solves the 1D root equation

\[ f(R) = \sum_k n_k\,\big[s_k - E_k(R)\big] = 0, \quad E_k(R) = \frac{1}{1 + 10^{\big(R^{\text{eff}}_{\text{opp},k} - R\big)/400}}. \]

Properties:

- \(f(R)\) is strictly decreasing in \(R\), so the root is unique when there are both non-trivial wins and losses.
- We solve for \(\hat R\) via robust 1D root finding (e.g., Brent’s method) on a bracket (\(\min R^{\text{eff}}_{\text{opp}} - 400, \max R^{\text{eff}}_{\text{opp}} + 400\)).

### Standard error and confidence intervals

The Fisher information for \(R\) under independent games with weights \(n_k\) is

\[ \mathcal{I}(R) = \sum_k n_k\,E_k(R)\,\big(1 - E_k(R)\big)\,\Big(\tfrac{\ln 10}{400}\Big)^2. \]

We estimate the standard error at \(\hat R\) as

\[ \operatorname{SE}(\hat R) = \frac{1}{\sqrt{\mathcal{I}(\hat R)}}. \]

A normal approximation yields a 95% CI \(\hat R \pm 1.96\,\operatorname{SE}\). Bootstrap (resampling games within each opponent block) is a robust alternative for small samples or extreme scores.

### Anchoring the rating scale

Elo is relative; we need anchors:

- Dragon engine levels: we map \(\text{lvl}=\ell\) to \(R = 125\,(\ell + 1)\) as a baseline.
- Random player: we estimate \(R_{\text{random}}\) by fitting its results vs Dragon, using the same MLE with color adjustment.
- Other engines (e.g., Stockfish skill levels) can be included if a mapping to Elo is provided.

Once anchors are established, we estimate an LLM’s Elo by combining all its games against anchored opponents.

### Using `win_rates.csv` in this repo

`data_processing/win_rates.csv` contains rows with columns:

- `player_white`, `player_black`, `games`, `win_rate_white`, `win_rate_black`, `draw_rate`, optional `player_white_elo`, `player_black_elo`.

To fit an unrated player \(P\):

1) For each row where \(P\) appears and the opponent has known Elo (either provided in the CSV or from an anchor mapping), compute the score fraction for \(P\) given the color:
   - If \(P\) is White: \(s = \text{win\_rate\_white}/100 + 0.5\,\text{draw\_rate}/100\).
   - If \(P\) is Black: \(s = \text{win\_rate\_black}/100 + 0.5\,\text{draw\_rate}/100\).
2) Use \(n = \text{games}\) as the weight.
3) Build records \((R^{\text{eff}}_{\text{opp}}, s, n)\) using the color adjustment above.
4) Solve the MLE for \(\hat R\) and compute \(\operatorname{SE}\) and a 95% CI.

Practical details used here:

- We canonicalize names (e.g., alias `random_player` → `random`) to match anchors.
- We deduplicate exact duplicate rows (some sources repeat the same pairing and rates).
- We can optionally group multiple rows of the same pairing by summing `games` and recomputing the score fraction.
- We enforce a minimum games threshold (e.g., \(n \ge 30\)) for reporting.

### Example: single-opponent inversion

Suppose a player as Black scored \(S=0.60\) vs an opponent of Elo \(R_{\text{opp}}=1000\) with \(\gamma=35\):

\[ \Delta = -400\,\log_{10}\!\left(\frac{1}{0.60} - 1\right) \approx 70.4, \quad R \approx 1000 + 70.4 + 35 = 1105.4. \]

### Example: multi-opponent MLE (concept)

If the same player faced three opponents with Elos \(\{800, 1000, 1200\}\) for \(\{40,50,60\}\) games and score fractions \(\{0.70,0.55,0.40\}\) as Black, then we solve

\[ \sum_{k=1}^3 n_k \left[s_k - \frac{1}{1 + 10^{(R_{\text{opp},k} + \gamma - R)/400}}\right] = 0 \]

for \(R\). The solution is unique and can be found with Brent’s method; we then compute \(\operatorname{SE}\) via Fisher information.

### Extensions and caveats

- **Minimum samples**: Require \(n\) per player or per opponent block (e.g., \(\ge 30\)).
- **Shrinkage**: Apply a weak Gaussian prior to pull small-\(n\) estimates toward a central value.
- **Bootstrap**: Resample outcomes within each (opponent, color) block to get nonparametric CIs.
- **Global fit**: For full use of LLM-vs-LLM games with draws and color, use a Bradley–Terry–Davidson model anchored by Dragon/Random Elos.
- **White advantage sensitivity**: Vary \(\gamma\) to assess robustness.

### Code references in this repo

- `data_processing/calc_elos_pipeline.py` implements the MLE solver (Brent root-finding), color adjustment, and staged reporting using Dragon and Random anchors.
- The same formulas are applied when estimating from the aggregated `win_rates.csv` after canonicalizing names and deduplicating rows.




In [60]:
# elos_from_win_rates.py
from pathlib import Path
from typing import List, Dict, Tuple
import math
import numpy as np
from scipy.optimize import root_scalar

WIN_RATES_CSV = Path("./win_rates.csv")
WHITE_ADV = 35.0  # White advantage in Elo

ALIASES = {
    "random_player": "random",
    "Random_Player": "random",
    # add other cosmetic aliases if needed
}

def canon(name: str) -> str:
    return ALIASES.get(name, name)

def logistic(x):
    return 1.0 / (1.0 + 10.0**(x / 400.0))

def expected_score(R_player, R_opp_eff):
    # σ(opp_eff - R)
    return logistic(R_opp_eff - R_player)

def estimate_from_aggregated(records: List[Tuple[float, float, int, str]], white_adv=WHITE_ADV):
    """
    records: list of tuples (opp_elo, s, n, color)
      - s in [0,1] is score fraction: win + 0.5*draw
      - n is number of games
      - color in {"white","black"} indicates the color of the player being estimated
    Solves sum_k n_k * [s_k - σ((opp_elo ± γ) - R)] = 0 for R.
    Returns (R_hat, se, n_total).
    """
    if not records:
        return float('nan'), float('nan'), 0

    opp_eff = []
    s = []
    w = []
    for opp_elo, s_k, n_k, color in records:
        adj = white_adv if color == "black" else -white_adv
        opp_eff.append(opp_elo + adj)
        s.append(s_k)
        w.append(n_k)
    opp_eff = np.array(opp_eff, dtype=float)
    s = np.array(s, dtype=float)
    w = np.array(w, dtype=float)

    def f(R):
        E = expected_score(R, opp_eff)
        return float(np.sum(w * (s - E)))

    a = float(opp_eff.min() - 800.0)
    b = float(opp_eff.max() + 800.0)
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        # expand bracket a bit
        for extra in (800, 1600, 3200):
            a2, b2 = a - extra, b + extra
            fa, fb = f(a2), f(b2)
            if fa * fb <= 0:
                a, b = a2, b2
                break
    if fa * fb > 0:
        return float('nan'), float('nan'), int(w.sum())

    root = root_scalar(f, bracket=(a, b), method="brentq").root
    Ehat = expected_score(root, opp_eff)
    info = np.sum(w * Ehat * (1.0 - Ehat)) * (math.log(10) / 400.0) ** 2
    se = 1.0 / math.sqrt(info) if info > 0 else float('nan')
    return float(root), float(se), int(w.sum())

def parse_rows(path: Path):
    seen = set()
    rows = []
    with path.open() as f:
        reader = csv.DictReader(f)
        for r in reader:
            r = dict(r)  # copy
            r["player_white"] = canon(r["player_white"])
            r["player_black"] = canon(r["player_black"])
            # dedupe across sources: keep unique pairing+rates
            key = (
                r["player_white"],
                r["player_black"],
                r["win_rate_white"],
                r["win_rate_black"],
                r["draw_rate"],
            )
            if key in seen:
                continue
            seen.add(key)
            rows.append(r)
    return rows

def to_float(x):
    try:
        return float(x) if x not in (None, "", "N/A") else None
    except:
        return None

def score_fraction(r: Dict, for_color: str) -> float:
    wr = float(r["win_rate_white"]) / 100.0
    br = float(r["win_rate_black"]) / 100.0
    dr = float(r["draw_rate"]) / 100.0
    return (wr + 0.5 * dr) if for_color == "white" else (br + 0.5 * dr)

def build_anchors(rows: List[Dict]) -> Dict[str, float]:
    anchors = {}
    # Prefer explicit Elo columns when provided
    for r in rows:
        pw, pb = r["player_white"], r["player_black"]
        ew = to_float(r.get("player_white_elo"))
        eb = to_float(r.get("player_black_elo"))
        if ew is not None:
            anchors[pw] = ew
        if eb is not None:
            anchors[pb] = eb
    # Fallback: parse dragon-lvl-N if needed
    for name in list(set([r["player_white"] for r in rows] + [r["player_black"] for r in rows])):
        if name.startswith("dragon-lvl-") and name not in anchors:
            try:
                lvl = int(name.split("-")[-1])
                anchors[name] = (lvl + 1) * 125.0
            except:
                pass
    return anchors

def records_for_player(rows: List[Dict], anchors: Dict[str, float], player: str):
    recs = []
    for r in rows:
        pw, pb = r["player_white"], r["player_black"]
        games = int(r["games"])
        if games <= 0:
            continue
        if pw == player and pb in anchors:
            s = score_fraction(r, "white")
            recs.append((anchors[pb], s, games, "white"))
        elif pb == player and pw in anchors:
            s = score_fraction(r, "black")
            recs.append((anchors[pw], s, games, "black"))
    return recs


rows = parse_rows(WIN_RATES_CSV)
anchors = build_anchors(rows)

# 1) Calibrate random if present
if "random" not in anchors and any(r["player_white"] == "random" for r in rows):
    rand_recs = records_for_player(rows, anchors, "random")
    R_rand, se_rand, n_rand = estimate_from_aggregated(rand_recs, WHITE_ADV)
    if not math.isnan(R_rand):
        anchors["random"] = R_rand
        print(f"Calibrated random: {R_rand:.1f} ± {1.96*se_rand:.1f} (n={n_rand})")

# 2) Estimate all non-anchored players who faced anchored opponents
results = {}
candidates = set([r["player_white"] for r in rows] + [r["player_black"] for r in rows])
for p in sorted(candidates):
    if p in anchors:
        continue
    recs = records_for_player(rows, anchors, p)
    if not recs:
        continue
    R, se, n = estimate_from_aggregated(recs, WHITE_ADV)
    results[p] = (n, R, 1.96 * se)

# Print a compact table, filter to reasonable sample sizes if desired
MIN_GAMES = 30
table = [(p, n, R, ci) for p, (n, R, ci) in results.items() if n >= MIN_GAMES and not math.isnan(R)]
table.sort(key=lambda x: x[2], reverse=True)
print(f"\nEstimated Elo from {WIN_RATES_CSV.name} (n >= {MIN_GAMES}):")
print(f"{'Player':<45} {'Games':>6} {'Elo':>8} {'±95%CI':>8}")
for p, n, R, ci in table:
    print(f"{p:<45} {n:>6d} {R:>8.1f} {ci:>8.1f}")


Calibrated random: -122.3 ± 23.0 (n=4000)

Estimated Elo from win_rates.csv (n >= 30):
Player                                         Games      Elo   ±95%CI
o3-2025-04-16-low                                363    773.2     51.7
gpt-5-nano-2025-08-07-high                       106    639.5     92.6
grok-3-mini-fast-beta-high                       106    590.8     71.6
gpt-5-2025-08-07-low                              43    562.0    344.5
o4-mini-2025-04-16-high_timeout-20m               59    439.5     89.0
o3-mini-2025-01-31-high                          129    438.8     63.5
gpt-5-nano-2025-08-07-medium                     132    413.3     77.5
o4-mini-2025-04-16-high_timeout-60m               39    401.1    109.1
gpt-5-mini-2025-08-07-high                        35    399.7    247.9
grok-3-mini-beta-high                            335    387.1     44.2
o4-mini-2025-04-16-high                          164    380.2     61.1
gpt-5-mini-2025-08-07-medium                      33    312.7